## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [1]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

In [2]:
def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

In [3]:
file_id = '1IpwgaYYmC5hmNuAq97KI-qQtEIpDoLwl'
download_file(file_id, 'data.zip')
file_id = '1mh6wtj72ij9npCM8dy8kW2H1wGuu6oxl'
download_file(file_id, 'test.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1IpwgaYYmC5hmNuAq97KI-qQtEIpDoLwl
From (redirected): https://drive.google.com/uc?id=1IpwgaYYmC5hmNuAq97KI-qQtEIpDoLwl&confirm=t&uuid=57a368da-0acd-4b58-a6af-d026ccad7812
To: /content/data.zip
100%|██████████| 289M/289M [00:06<00:00, 42.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mh6wtj72ij9npCM8dy8kW2H1wGuu6oxl
To: /content/test.zip
100%|██████████| 7.18M/7.18M [00:00<00:00, 22.0MB/s]


In [4]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[:-4]}/')

In [5]:
dataset_extract('data.zip')
dataset_extract('test.zip')

Extracting: 100%|██████████| 21/21 [00:00<00:00, 932.75files/s]


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [6]:
#test,train,valid 폴더 생성
import os
path = '/content/cool_roof/'

for folder in ['train','test','valid']:
    os.makedirs(path+folder+'/images',exist_ok=True)
    os.makedirs(path+folder+'/labels',exist_ok=True)

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [8]:
import cv2
import numpy as np

def rgb_to_gray_norm(image_path,save_path=''):
    # RGB 이미지를 그레이스케일로 변환
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # 그레이스케일 이미지를 min-max 스케일링
    scaled_image = (gray_image - np.min(gray_image)) / (np.max(gray_image) - np.min(gray_image))
    scaled_image = scaled_image * 255
    scaled_image = scaled_image.astype(np.uint8)

    # 스케일링 된 그레이 이미지를 yolo에 입력하기위해 rgb scale로 변환
    scaled_rgb_image = cv2.cvtColor(scaled_image, cv2.COLOR_GRAY2RGB)

    if save_path != '':
        if save_path[-1] !='/':
            save_path = save_path+'/'
    save_path = save_path+image_path.split('/')[-1]

    cv2.imwrite(save_path, scaled_rgb_image)


In [10]:
rgb_to_gray_norm('/content/data/╗⌡ ╞·┤⌡/train/images/-2024-04-12-092904_mp4-0000_jpg.rf.29f0da09fada403f53585d9928a4250d.jpg')

In [11]:
from glob import glob
from sklearn.model_selection import train_test_split

image_path = glob('/content/data/╗⌡ ╞·┤⌡/train/images/*')
len(image_path)

716

In [12]:
train_img, valid_img, _, _ = train_test_split(image_path,image_path,test_size=0.2,random_state=2024)

len(train_img),len(valid_img)

(572, 144)

In [13]:
train_img[1].split('/')[-1][:-3]+'txt'

'roof (1).'

In [14]:
train_img[1].replace('cool_roof_images','cool_roof_yolo_labels')[:-3]+'txt'

'/content/data/╗⌡ ╞·┤⌡/train/images/roof (1).txt'

In [15]:
# train 파일 이동
for img_path in train_img:
    label_path = '/content/data/╗⌡ ╞·┤⌡/train/labels/' + img_path.split('/')[-1][:-3]+'txt'
    if os.path.exists(img_path):
        rgb_to_gray_norm(img_path,path+'train/images')
    if os.path.exists(label_path):
        shutil.move(label_path,path+'train/labels')
# valid 파일 이동
for img_path in valid_img:
    label_path = '/content/data/╗⌡ ╞·┤⌡/train/labels/' + img_path.split('/')[-1][:-3]+'txt'
    if os.path.exists(img_path):
        rgb_to_gray_norm(img_path,path+'valid/images')
    if os.path.exists(label_path):
        shutil.move(label_path,path+'valid/labels/')
# test 파일 이동
for img_path in glob('/content/test/test/*'):
    if os.path.exists(img_path):
        rgb_to_gray_norm(img_path,path+'test/images')

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [16]:
import yaml

yaml_data = {'train' : '/content/cool_roof/train/images',
               'val' : '/content/cool_roof/valid/images',
               'test' : '/content/cool_roof/test/images',
               'names' : ['cool roof', 'generic roof'],
               'nc' : 2 }


yaml_file_path = "/content/cool_roof/data.yaml"
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(yaml_data, yaml_file)

### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [17]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 11.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [18]:
from ultralytics import settings, YOLO

In [19]:
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/datasets',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [20]:
settings['datasets_dir'] = '/content/'
settings.update()
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [25]:
model_transfer = YOLO(model='yolov8n.pt', task='detect')

In [26]:
model_transfer.train(data='/content/cool_roof/data.yaml',
                    epochs=100,
                    patience=10,
                    seed=2024,
                    pretrained=False,
                    hsv_h=0,
                    hsv_s=0,
                    hsv_v=0,
                    )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/cool_roof/data.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

train: Scanning /content/cool_roof/train/labels.cache... 572 images, 0 backgrounds, 1 corrupt: 100%|██████████| 572/572 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/cool_roof/train/images/KakaoTalk_20240412_100143804.mp4_20240412100854_0168.PNG: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0194]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/cool_roof/valid/labels.cache... 144 images, 0 backgrounds, 0 corrupt: 100%|██████████| 144/144 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        144       1048      0.588      0.587       0.59      0.359
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 25, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



35 epochs completed in 0.072 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:07<00:00,  1.47s/it]


                   all        144       1048      0.616      0.635      0.674      0.416
             cool roof        144        222      0.558      0.557      0.581      0.356
          generic roof        144        826      0.675      0.713      0.766      0.476
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 24.9ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7849100b8790>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [27]:
image_path = '/content/cool_roof/test/images'
model_transfer.predict(source=image_path, save=True, iou=0.5, conf=0.5)


image 1/21 /content/cool_roof/test/images/KakaoTalk_20240411_143843284.png: 640x576 3 cool roofs, 13 generic roofs, 11.4ms
image 2/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 112419.png: 576x640 3 generic roofs, 11.5ms
image 3/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 112543.png: 608x640 (no detections), 11.3ms
image 4/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 112738.png: 640x640 3 generic roofs, 9.9ms
image 5/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 112832.png: 576x640 2 generic roofs, 9.3ms
image 6/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 112921.png: 640x640 1 cool roof, 3 generic roofs, 9.6ms
image 7/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 113028.png: 608x640 2 cool roofs, 12 generic roofs, 9.6ms
image 8/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 113048.png: 576x640 2 cool roofs, 5 generic roofs, 9.3ms
image 9/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 113117.png: 640x640 1 generic roof,

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'cool roof', 1: 'generic roof'}
 obb: None
 orig_img: array([[[  6,   6,   6],
         [  6,   6,   6],
         [  7,   7,   7],
         ...,
         [ 28,  28,  28],
         [ 37,  37,  37],
         [ 42,  42,  42]],
 
        [[  7,   7,   7],
         [  8,   8,   8],
         [  9,   9,   9],
         ...,
         [ 33,  33,  33],
         [ 40,  40,  40],
         [ 45,  45,  45]],
 
        [[ 15,  15,  15],
         [ 15,  15,  15],
         [ 16,  16,  16],
         ...,
         [ 33,  33,  33],
         [ 40,  40,  40],
         [ 44,  44,  44]],
 
        ...,
 
        [[167, 167, 167],
         [164, 164, 164],
         [164, 164, 164],
         ...,
         [ 25,  25,  25],
         [ 18,  18,  18],
         [ 13,  13,  13]],
 
        [[165, 165, 165],
         [163, 163, 163],
         [163, 163, 163],
         .

In [28]:
model_transfer.export(format='torchscript')

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'runs/detect/train3/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)

TorchScript: starting export with torch 2.2.1+cu121...
TorchScript: export success ✅ 1.4s, saved as 'runs/detect/train3/weights/best.torchscript' (11.9 MB)

Export complete (2.7s)
Results saved to /content/runs/detect/train3/weights
Predict:         yolo predict task=detect model=runs/detect/train3/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train3/weights/best.torchscript imgsz=640 data=/content/cool_roof/data.yaml  
Visualize:       https://netron.app


'runs/detect/train3/weights/best.torchscript'